In [1]:
import numpy as np
import pandas as pd

## Conjuntos 
Del modelo de optimización

| **Término** | **Descripción**                |
|-------------|--------------------------------|
| $I$         | Conjunto de productores        |
| $J$         | Conjunto de centros de acopio  |
| $K$         | Conjunto de clientes           |
| $P$         | Conjunto de productos          |
| $T$         | Conjunto de periodos de tiempo |

In [2]:
# I = 'Productores'
J = 'CAcopios'
K = 'Clientes'
P = 'Productos'
# T = 'Tiempo'

In [3]:
csv_base = './csv/'

routes = {
    # Capacidades de centros de acopio
    'cic': 'CapInvCA.csv',
    'cictest': 'CapInvCAtest.csv',
    # Costo fijo del centro de acopio
    'cfa': 'CostoFijoCA.csv',
    'cfatest': 'CostoFijoCAtest.csv',
    # Costo variable del centro de acopio
    'cva' : 'CostoVarCA.csv',
    # Costo de inventario del centro de acopio
    'cia' : 'CostoInvCA.csv',
    'ciatest' : 'CostoInvCAtest.csv',
    # Costo de transporte del centro de acopio al cliente
    'ctac' : 'CostoTransCAClie.csv',
    'ctactest' : 'CostoTransCAClietest.csv'
    
}

csv_config = {
    'delimiter' : ';',
    'decimal' : ','
}

# pd.read_csv(csv_base + routes['cfa'], **csv_config)

## Centros de acopio

In [4]:
J_df = pd.read_csv(csv_base + J + 'test' + '.csv')
J_df

,CAcopios
0,San Juan del Cesar
1,Riohacha
2,Maicao


## Clientes

In [5]:
K_df = pd.read_csv(csv_base + K + 'test' + '.csv')
K_df

,Clientes
0,Distribuidor1
1,Distribuidor2


## Productos

In [6]:
P_df = pd.read_csv(csv_base + P + '.csv')
P_df.head()

,Productos
0,Queso duro
1,Queso blando


# Escenarios de prueba
Tomando demandas por cada centro de acopio, se definen distintos escenarios de posibles casos.

**Escenario 1**: Un centro de acopio sobrepasa la demanda del cliente
- El modelo puede asignar dicho centro de acopio para suplir con la demanda
- El centro de acopio debe proveer esa demanda

**Escenario 2**: Ningún centro de acopio cumple con toda la demanda del cliente
- El modelo debe dar una solución de varios centros de acopio
- La suma de la demanda por cada centro de acopio debe ser igual a la demanda

**Escenario 3**: Un centro de acopio tiene en stock exactamente la cantidad de la demanda de un cliente.
- El modelo puede tomar el centro de acopio que tiene en stock exactamente la demanda a cumplir

### Demandas
Generando demandas para cada cliente, se usarán como entrada para el modelo

In [7]:
clientes = K_df[K].drop_duplicates()

demandas = []

for client in clientes:
    demandas.append(20 * np.random.randint(10))

clientes = pd.DataFrame(clientes)

clientes['Demanda'] = pd.DataFrame(demandas)
clientes

,Clientes,Demanda
0,Distribuidor1,60
1,Distribuidor2,140


### Capacidades 
Obteniendo las capacidades de cada centro de acopio

In [8]:
cap_df = pd.read_csv(csv_base + routes['cictest'], **csv_config)

cap_df

,CAcopios,Productos,CapInvCA
0,San Juan del Cesar,Queso duro,50
1,San Juan del Cesar,Queso blando,320
2,Riohacha,Queso duro,60
3,Riohacha,Queso blando,90
4,Maicao,Queso duro,100
5,Maicao,Queso blando,0


## Función objetivo
$$
Min(F) = CProduccion_t + COperacion_t + CInventario_t + CTransporte_t
$$

#### Costos de producción
$$
  CProduccion_t = \sum_{i \in I} \sum_{j \in J} \sum_{p \in P} CostoProduccion_{pit} PA_{pijt} \qquad \forall_t \in T
$$

In [9]:
# Costo producción
# cp_df = pd.read_csv(csv_base + routes['cp'], **csv_config)
# cp_df.head()

#### Costos de Operación
$$
  COperacion_t = CFijos_t + CVariables_t
$$

In [10]:
cfa_df = pd.read_csv(csv_base + routes['cfatest'], **csv_config)
cfa_df.head()

,CAcopios,Productos,CostoFijoCA
0,San Juan del Cesar,Queso duro,22000
1,San Juan del Cesar,Queso blando,25000
2,Riohacha,Queso duro,30000
3,Riohacha,Queso blando,31000
4,Maicao,Queso duro,28000


In [11]:
# Costo variable
# cva_df = pd.read_csv(csv_base + routes['cva'], **csv_config)
# cva_df

#### Costos de Inventario
$$
  CInventario_t = \sum_{j \in J} \sum_{p \in P} CostoInvAcopio_{pjt} InvCA_{pjt} \qquad \forall_t \in T
$$

In [12]:
cia_df = pd.read_csv(csv_base + routes['ciatest'], **csv_config)
cia_df.head()

,CAcopios,Productos,CostoInvCA
0,San Juan del Cesar,Queso duro,1000
1,San Juan del Cesar,Queso blando,1500
2,Riohacha,Queso duro,500
3,Riohacha,Queso blando,800
4,Maicao,Queso duro,900


#### Costos de Transporte
$$
  CTransporte_t = \sum_{j \in J} \sum_{i \in I} \sum_{p \in P} CostoTransAcopio_{pijt} PA_{pijt} + \sum_{k \in K} \sum_{j \in J} \sum_{p \in P} CostoTransAcopioClie_{pjkt} \qquad \forall_t \in T
$$

In [13]:
# Costo productor-acopio
# ctpa_df = pd.read_csv(csv_base + routes['ctpa'], **csv_config)
# ctpa_df

In [14]:
ctac_df = pd.read_csv(csv_base + routes['ctactest'], **csv_config)
ctac_df.head()

,CAcopios,Clientes,CostoTransCAClie
0,San Juan del Cesar,Distribuidor1,68000
1,San Juan del Cesar,Distribuidor2,38000
2,Riohacha,Distribuidor1,21000
3,Riohacha,Distribuidor2,15000
4,Maicao,Distribuidor1,99000


## Esquema de representación
Se recibe la demanda $DemandaClie_{pkt}$ como parámetro de entrada.

Se representa la cantidad $n$ de centros de acopio como un vector, donde se asigna una cantidad $x_i$ para cada centro de acopio $J_i$, la cantidad asignada representa la demanda que va a ser suplida por ese centro de acopio.
$$
\begin{array} {|r|r|r|r|r|r|}
    \hline x_0 & x_1 & x_2 & x_3 & \cdots & x_n \\
    \hline
\end{array}
\quad \therefore \quad x_i = AC_{pjkt}
$$
Donde se aplican las restricciones:
$$
\begin{align*}
    \sum_{i=0}^{n} x_i &= DemandaClie_{pkt} \\
    x_i &\leq InvCA_{pjt}
\end{align*}
$$

In [15]:
X = np.zeros(J_df.shape)
dem = clientes.loc[0, 'Demanda']

cap_queso = cap_df[cap_df[P] == 'Queso duro']

for i in range(X.shape[0]):
    rand = np.random.randint(dem)
    cap = cap_df.loc[i, 'CapInvCA']
        
    if i == X.size - 1 and dem != 0:
        X[i] = dem
        dem -= dem
    else:
        if rand > cap:
            X[i] = cap
            dem -= cap
        else:
            X[i] = rand
            dem -= rand
    
suma = X.sum()
    
print(X)
print(f'Suma de la distribución: {suma}')

[[50.]
 [ 9.]
 [ 1.]]
Suma de la distribución: 60.0


In [16]:
def costos():
    sum_costos_ca = []

    for costo in range(X.shape[0]):
        ca = J_df.loc[costo]
        
        if X[costo] == 0:
            sum_costos_ca.append((ca.tolist()[0], 0.))
            continue
    
        costo_fijo_ca = cfa_df[(cfa_df[J] == ca[J]) & (cfa_df[P] == 'Queso duro')]
        costo_inv_ca = cia_df[(cia_df[J] == ca[J]) & (cia_df[P] == 'Queso duro')]
        costo_trans_ca = ctac_df[(ctac_df[J] == ca[J]) & (ctac_df[K] == 'Distribuidor1')]
    
        delta = X[costo] + costo_fijo_ca['CostoFijoCA']
        delta += X[costo] * costo_inv_ca['CostoInvCA']
        delta += np.array(costo_trans_ca['CostoTransCAClie'])
    
        sum_costos_ca.append((ca.tolist()[0], delta.tolist()[0]))
        
    return sum_costos_ca

costos()

[('San Juan del Cesar', 140050.0), ('Riohacha', 55509.0), ('Maicao', 127901.0)]

## scipy

In [17]:
from scipy.optimize import dual_annealing
# from scipy.optimize import differential_evolution

from scipy.optimize import LinearConstraint

def f(x):
    delta = 0
    print(f'{x} -> Suma: {x.sum()}')
    
    for idx in range(x.shape[0]):
        ca = J_df.loc[idx]
    
        if x[idx] == 0.:
            continue
    
        costo_fijo_ca = cfa_df[(cfa_df[J] == ca[J]) & (cfa_df[P] == 'Queso duro')]
        costo_inv_ca = cia_df[(cia_df[J] == ca[J]) & (cia_df[P] == 'Queso duro')]
        costo_trans_ca = ctac_df[(ctac_df[J] == ca[J]) & (ctac_df[K] == 'Distribuidor1')]
    
        delta += np.array(costo_fijo_ca['CostoFijoCA'].values[0])
        delta += x[idx] * costo_inv_ca['CostoInvCA'].values[0]
        delta += np.array(costo_trans_ca['CostoTransCAClie'].values[0])
        
    print(f'{delta}\n')
    
    return delta


bounds = [(0, x_i) for x_i in cap_queso['CapInvCA']]
cons = {'constraints': LinearConstraint(np.ones(X.shape[0]), suma, suma)}
x0 = [x[1] for x in costos()]

result = dual_annealing(
    f,
    bounds=bounds,
    minimizer_kwargs=cons,
    maxiter=2,
    x0=np.array(x0)
)
print(result)

[140050.  55509. 127901.] -> Suma: 323460.0
283183400.0

[22.29601477 49.12556177 41.12556177] -> Suma: 112.54713831841946
351871.80125415325

[20.38167553 39.11094832 51.11094832] -> Suma: 110.6035721655935
353937.0031738654

[22.28241508 20.4109744  92.4109744 ] -> Suma: 135.10436388663948
383657.7792437747

[ 0.28401566 49.12556177 41.12556177] -> Suma: 90.53513921052217
329859.80214625597

[ 0.28401566 37.15374976 41.12556177] -> Suma: 78.56332720071077
323873.89614135027

[ 0.28401566 37.15374976 66.49550525] -> Suma: 103.93327067792416
346706.8452708423

[ 0.28401566 37.15374976 41.12556177] -> Suma: 78.56332720071077
323873.89614135027

[ 0.28401568 37.15374976 41.12556177] -> Suma: 78.56332721561193
323873.89615625143

[ 0.28401566 37.15374978 41.12556177] -> Suma: 78.56332721561193
323873.89614880085

[ 0.28401566 37.15374976 41.12556179] -> Suma: 78.56332721561193
323873.8961547613

[-205.90376007  330.96597403  -65.06221396] -> Suma: 60.00000000000004
169023.23438078162

[-2